In [9]:
import pandas as pd
import numpy as np
import math


SyntaxError: invalid syntax (1439492683.py, line 4)

In [3]:
generalized_data_gt_50k = pd.read_csv('hw1-1-generalized_data_gt_50k_3_3_4_5.csv')
generalized_data_le_50k = pd.read_csv('hw1-1-generalized_data_le_50k_2_3_4_5.csv')


In [6]:
def calculate_entropy(group):
    """ Calculate entropy for a single group of records """
    if len(group) == 0:
        return 0
    value_counts = group.value_counts()
    probabilities = value_counts / value_counts.sum()
    entropy = -np.sum(probabilities * np.log(probabilities))
    return entropy

def check_l_diversity(data, l):
    """ Check if the dataset satisfies l-diversity """
    entropy_threshold = math.log(l)  # Define the minimum entropy threshold
    fails = 0  # Track number of failures
    
    # Group data by QIs and calculate entropy for each group
    grouped = data.groupby(['age', 'education', 'marital_status', 'race'])
    for name, group in grouped:
        entropy = calculate_entropy(group['occupation'])
        if entropy < entropy_threshold:
            print(f'Group {name} fails to meet the entropy l-diversity with entropy {entropy:.4f}')
            fails += 1
    if fails == 0:
        print("All groups meet the l-diversity requirement.")
        return True
    else:
        print(f"{fails} groups fail to meet the l-diversity requirement.")
        return False

l = 3
print("Checking >50K dataset for l-diversity:")
satisfied_l_diversity_gt_50k = check_l_diversity(generalized_data_gt_50k, l)


Checking >50K dataset for l-diversity:
Group ('All Ages', 'Advanced Degree', 'Married', 'Non-Western Origin') fails to meet the entropy l-diversity with entropy 1.0453
Group ('All Ages', 'Advanced Degree', 'Not-Married', 'Non-Western Origin') fails to meet the entropy l-diversity with entropy 0.6829
2 groups fail to meet the l-diversity requirement.


In [7]:

print("Checking <=50K dataset for l-diversity:")
satisfied_l_diversity_le_50k = check_l_diversity(generalized_data_le_50k, l)

Checking <=50K dataset for l-diversity:
All groups meet the l-diversity requirement.


In [11]:
# Will need this function to generalize education levels in the next step
def generalize_education(education, level):
    education = education.strip()
    
    if level == 1:
        return education
    
    elif level == 2:
        if education in ['Preschool', '1st-4th', '5th-6th']:
            return 'Primary School'
        elif education in ['7th-8th', '9th']:
            return 'Middle School'
        elif education in ['10th', '11th', '12th', 'HS-grad']:
            return 'High School'
        elif education in ['Assoc-voc', 'Prof-school', 'Some-college']:
            return 'Vocational School'
        elif education in ['Assoc-acdm', 'Bachelors']:
            return 'Undergraduate School'
        elif education in ['Masters', 'Doctorate']:
            return 'Graduate School'
    
    elif level == 3:
        if education in ['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th', 'HS-grad']:
            return 'Compulsory School'
        elif education in ['Assoc-voc', 'Prof-school', 'Some-college', 'Assoc-acdm', 'Bachelors']:
            return 'Basic Degree'
        elif education in ['Masters', 'Doctorate']:
            return 'Advanced Degree'
    
    elif level == 4:
        if education == 'Compulsory School':
            return 'Compulsory School'
        elif education in ['Basic Degree', 'Advanced Degree']:
            return 'Advanced School'
    
    elif level == 5:
        return 'Education'

    return 'Education'



Increase the generalization level for a specific attribute, say 'education'

In [13]:
def adjust_generalization(data):
    # Modify the generalize_education function or call it with a higher level for specific groups
    data['education'] = data.apply(lambda row: generalize_education(row['education'], 4) if row['race'] == 'Non-Western Origin' else row['education'], axis=1)
    return data

# Apply adjusted generalization
generalized_data_gt_50k_adjusted = adjust_generalization(generalized_data_gt_50k.copy())

# Check l-diversity again
print("Rechecking >50K dataset for l-diversity after adjustment:")
satisfied_l_diversity_gt_50k_adjusted = check_l_diversity(generalized_data_gt_50k_adjusted, l)
# Save the adjusted dataset
generalized_data_gt_50k_adjusted.to_csv('hw1-1-generalized_data_gt_50k_adjusted_for_lDiversity.csv', index=False)


Rechecking >50K dataset for l-diversity after adjustment:
All groups meet the l-diversity requirement.
